In [2]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
import re

In [3]:
def word_occurence_matrix(text, target=None, stop_words=None, binary=True, preprocess_text=False):  
    '''
    Output is messages x (unique) words
    
    If binary=True, then each element represents if the word is in the message or not.
    Otherwise, it represents the count of how many times that word appears in that message.
    ''' 
    if target:
        text = list(filter(lambda x : target in x, text)) #Filter comments in which target word is present
        
    preprocessor = CountVectorizer(strip_accents='unicode').build_preprocessor()   
    if stop_words:        
        stop_words = [preprocessor(word) for word in stop_words] #preprocesses stop words
    if preprocess_text:
        text = [preprocessor(msg) for msg in text] #preprocesses text
        
    #calculates word count for each message
    vectorizer = CountVectorizer(strip_accents='unicode', stop_words=stop_words, binary=binary)
    X = vectorizer.fit_transform(text).toarray()
    
    labels = vectorizer.get_feature_names()
    
    return X, labels

### Dados

In [4]:
comments = pd.read_csv('../../dados/youtube/comentarios1.csv')
stop_words = [word.rstrip() for word in open('../stopwords.txt')]

preprocessor = CountVectorizer(strip_accents='unicode').build_preprocessor() 
#lowercase and strip accents
stop_words = [preprocessor(word) for word in stop_words]
#English and Spanish Words
stop_words.extend(['you', 'good', 'the', 'to', 'live', 'very', 'your', 'work', 'is', 'my', 'from', 'love', 'and', 
                   'in', 'thank', 'informative', 'are', 'of', 'un', 'english', 'what', 'mi', 'hello', 'el', 
                   'but', 'doctor'])
#Nomes Próprios
stop_words.extend(['drauzio', 'varella', 'lucy', 'kerr', 'julio', 'pereira','fernando', 'pinto', 'gomes', 'lair', 
                   'ribeiro', 'alvaro', 'galvao', 'felipe', 'ades', 'gomez', 'alain', 'dutra'])
#Pronomes e preposições
stop_words.extend(['pra', 'vc', 'todos', 'tudo', 'cada', 'nada', 'sobre'])
#Conjunções
stop_words.extend(['porque', 'pois', 'pq'])
#Advérbios
stop_words.extend(['assim', 'bem', 'ainda', 'agora', 'sim', 'sempre', 'aqui', 'la', 'tbm', 'ai', 'hoje'])
#Verbos frequentes
stop_words.extend(['vai', 'ser', 'ter', 'ta', 'fazer', 'fiz', 'faz', 'vou'])
#Outros
stop_words.extend(['boa', 'bom', 'obrigado', 'ola'])

#Remove urls
comments['text'] = [re.sub(r'http\S+', '', msg) for msg in comments['text']]
#Remove emails
comments['text'] = [re.sub(r'\S*@\S*\s?', '', msg) for msg in comments['text']]
#lowercase and strip accents
comments['text'] = [preprocessor(msg) for msg in comments['text']]
#remove stopwords and punctuation
comments['text'] = [' '.join([word for word in RegexpTokenizer(r'\w+').tokenize(msg) if not word in stop_words])
                    for msg in comments['text']]
# Remove new line characters
comments['text'] = [re.sub(r'\s+', ' ', msg) for msg in comments['text']]
# Remove distracting single quotes
comments['text'] = [re.sub(r"\'", "", msg) for msg in comments['text']]
#Remove special characters
comments['text'] = [re.sub(r'([^a-zA-Z0-9\s]+?)', '', msg) for msg in comments['text']]

#Remove words with freq == 1
fdist = FreqDist(RegexpTokenizer(r'\w+').tokenize(' '.join(comments['text'])))
freq_filter = filter(lambda x: x[1] == 1, fdist.items())
fdist = [t[0] for t in freq_filter]
comments['text'] = [' '.join([word for word in RegexpTokenizer(r'\w+').tokenize(msg) if not word in fdist])
                    for msg in comments['text']]

N = 30 #Consider only the N most frequent words

In [5]:
comments_by_channel = []
channels = []
for channel, group in comments.groupby('uploader'):
    channel_comments = ' '.join(group['text'])
    comments_by_channel.append(channel_comments)
    channels.append(channel)

### Frequencia relativa

#### 1. Em relação ao total
Não usar essa. Deixei aqui por precaução.

Aqui foi usada a distribuição de probabilidade conjunta das palavras e canais, i.e., $p(a_i, c_j) = P(A=a_i, C= C_j)$.

In [5]:
X, labels = word_occurence_matrix(comments_by_channel, stop_words=stop_words, binary=False)

X = X / X.sum()

freq_lists = [list(zip(['%.4f'%(freq) for freq in row],labels)) for row in X]
freq_lists = [sorted(channel, reverse=True) for channel in freq_lists]
df_freq = pd.DataFrame(freq_lists, index = channels).T

df_freq.head(10)

,Dr. Alain Dutra,Dr. Alvaro Galvão,Dr. Felipe Ades MD PhD,Dr. Fernando Gomes,Dr. Lair Ribeiro Oficial,Drauzio Varella,Julio Pereira - Neurocirurgião,Lucy Kerr
0,"(0.0023, dias)","(0.0007, dias)","(0.0001, sintomas)","(0.0003, dr)","(0.0007, dr)","(0.0035, dr)","(0.0061, dias)","(0.0025, ivermectina)"
1,"(0.0017, tomar)","(0.0005, ivermectina)","(0.0001, dor)","(0.0001, parabens)","(0.0003, virus)","(0.0032, virus)","(0.0042, deus)","(0.0018, dias)"
2,"(0.0016, ivermectina)","(0.0004, tomar)","(0.0001, dias)","(0.0001, noite)","(0.0002, deus)","(0.0032, video)","(0.0042, cheiro)","(0.0016, dra)"
3,"(0.0013, dr)","(0.0004, dr)","(0.0001, dia)","(0.0000, zzzzzz)","(0.0001, video)","(0.0028, pessoas)","(0.0041, olfato)","(0.0013, tomar)"
4,"(0.0012, comprimidos)","(0.0004, comprimidos)","(0.0001, deus)","(0.0000, zz)","(0.0001, verdade)","(0.0020, casa)","(0.0040, paladar)","(0.0011, deus)"
5,"(0.0010, deus)","(0.0004, 15)","(0.0001, deu)","(0.0000, zumbis)","(0.0001, ver)","(0.0017, pode)","(0.0032, dor)","(0.0010, covid)"
6,"(0.0009, pode)","(0.0003, dose)","(0.0001, covid)","(0.0000, zumbido)","(0.0001, sistema)","(0.0017, dia)","(0.0027, sinto)","(0.0009, comprimidos)"
7,"(0.0009, dia)","(0.0002, pode)","(0.0000, zzzzzz)","(0.0000, zumbi)","(0.0001, senhor)","(0.0016, senhor)","(0.0026, sentir)","(0.0008, pode)"
8,"(0.0007, dra)","(0.0002, pessoas)","(0.0000, zz)","(0.0000, zuera)","(0.0001, saude)","(0.0016, dias)","(0.0026, gosto)","(0.0008, dr)"
9,"(0.0007, dose)","(0.0002, parabens)","(0.0000, zumbis)","(0.0000, zueira)","(0.0001, sabe)","(0.0016, deus)","(0.0025, sintomas)","(0.0008, dia)"


In [6]:
#Selects only the N with higher std
word_std = X.std(axis=0)
higher_std_zipped = sorted(zip(word_std, labels, X.T), reverse=True)[:N]
word_std, labels, X_t = zip(*higher_std_zipped)

word_std_i = ['%.4f'%(x) for x in word_std]
print(f'{N} palavras com maior desvio padrão entre os canais:\n')
print(list(zip(labels, word_std_i)))

30 palavras com maior desvio padrão entre os canais:

[('dias', '0.0019'), ('cheiro', '0.0014'), ('olfato', '0.0013'), ('paladar', '0.0013'), ('deus', '0.0013'), ('dr', '0.0010'), ('virus', '0.0010'), ('dor', '0.0010'), ('video', '0.0010'), ('pessoas', '0.0009'), ('sinto', '0.0009'), ('ivermectina', '0.0009'), ('sentir', '0.0009'), ('gosto', '0.0008'), ('dia', '0.0008'), ('sintomas', '0.0008'), ('covid', '0.0007'), ('casa', '0.0006'), ('febre', '0.0006'), ('pode', '0.0006'), ('cabeca', '0.0006'), ('tomar', '0.0006'), ('dra', '0.0005'), ('senhor', '0.0005'), ('medico', '0.0005'), ('falta', '0.0005'), ('fiquei', '0.0005'), ('doutor', '0.0005'), ('ar', '0.0005'), ('comprimidos', '0.0004')]


#### 2. Em relação ao canal
Foi obtida a distribuição de probabilidade das palavras em cada canal $C_k$, i.e., $p(a_i|C_k) = P(A=a_i|C_k)$. 
Com isso, podemos fazer corretamente comparações entre os canais. Foram, então, selecionadas as palavras cuja ocorrência mais se difere entre eles.

In [6]:
X, labels = word_occurence_matrix(comments_by_channel, stop_words=stop_words, binary=False)
#Makes frequencies relative to each channel so that we have the probability distribution of words for each channel
X = (X.T / X.sum(axis=1)).T

freq_lists = [list(zip(['%.4f'%(freq) for freq in row],labels)) for row in X]
freq_lists = [sorted(channel, reverse=True) for channel in freq_lists]
df_freq = pd.DataFrame(freq_lists, index = channels).T

print('Palavras com maior frequência relativa em cada canal:')
df_freq.head(10)

Palavras com maior frequência relativa em cada canal:


,Dr. Alain Dutra,Dr. Alvaro Galvão,Dr. Felipe Ades MD PhD,Dr. Fernando Gomes,Dr. Lair Ribeiro Oficial,Drauzio Varella,Julio Pereira - Neurocirurgião,Lucy Kerr
0,"(0.0195, dias)","(0.0201, dias)","(0.0288, dias)","(0.0616, dr)","(0.0272, dr)","(0.0097, dr)","(0.0210, dias)","(0.0158, ivermectina)"
1,"(0.0141, tomar)","(0.0148, ivermectina)","(0.0164, dia)","(0.0189, noite)","(0.0108, virus)","(0.0086, virus)","(0.0143, deus)","(0.0112, dias)"
2,"(0.0133, ivermectina)","(0.0138, dr)","(0.0164, covid)","(0.0186, parabens)","(0.0070, deus)","(0.0086, video)","(0.0143, cheiro)","(0.0101, dra)"
3,"(0.0114, dr)","(0.0135, 15)","(0.0158, sintomas)","(0.0071, lindo)","(0.0056, saude)","(0.0077, pessoas)","(0.0142, olfato)","(0.0080, tomar)"
4,"(0.0101, comprimidos)","(0.0129, comprimidos)","(0.0152, dor)","(0.0068, dia)","(0.0050, video)","(0.0054, casa)","(0.0138, paladar)","(0.0070, deus)"
5,"(0.0084, deus)","(0.0125, tomar)","(0.0142, deu)","(0.0061, deus)","(0.0048, pessoas)","(0.0047, dia)","(0.0108, dor)","(0.0065, covid)"
6,"(0.0077, dia)","(0.0079, dose)","(0.0124, deus)","(0.0059, obrigada)","(0.0045, mundo)","(0.0045, senhor)","(0.0091, sinto)","(0.0056, comprimidos)"
7,"(0.0076, pode)","(0.0068, kg)","(0.0108, positivo)","(0.0054, doutor)","(0.0045, medico)","(0.0045, pode)","(0.0090, sentir)","(0.0054, dr)"
8,"(0.0062, dose)","(0.0067, parabens)","(0.0096, teste)","(0.0050, amor)","(0.0042, covid)","(0.0045, dias)","(0.0088, gosto)","(0.0053, dia)"
9,"(0.0061, covid)","(0.0063, pode)","(0.0090, dr)","(0.0042, excelente)","(0.0039, china)","(0.0042, deus)","(0.0084, sintomas)","(0.0051, pode)"


In [7]:
#Selects only the N with higher std
word_std = X.std(axis=0)
higher_std_zipped = sorted(zip(word_std, labels, X.T), reverse=True)[:N]
word_std, labels, X_t = zip(*higher_std_zipped)

word_std_i = ['%.4f'%(x) for x in word_std]
print(f'{N} palavras com maior desvio padrão entre os canais:\n')
print(list(zip(labels, word_std_i)))

30 palavras com maior desvio padrão entre os canais:

[('dr', '0.0181'), ('dias', '0.0096'), ('ivermectina', '0.0069'), ('noite', '0.0057'), ('parabens', '0.0054'), ('dor', '0.0054'), ('tomar', '0.0051'), ('comprimidos', '0.0049'), ('olfato', '0.0049'), ('sintomas', '0.0048'), ('paladar', '0.0047'), ('cheiro', '0.0046'), ('deu', '0.0043'), ('covid', '0.0043'), ('15', '0.0042'), ('dia', '0.0038'), ('positivo', '0.0034'), ('dra', '0.0034'), ('deus', '0.0032'), ('sinto', '0.0031'), ('dose', '0.0030'), ('teste', '0.0030'), ('virus', '0.0029'), ('sentir', '0.0029'), ('gosto', '0.0028'), ('exame', '0.0027'), ('kg', '0.0026'), ('cabeca', '0.0025'), ('video', '0.0024'), ('lindo', '0.0023')]


#### 2.1. Por gênero

In [8]:
genero_df = pd.read_csv('../../dados/youtube/comentarios1_genero_90-100.csv', usecols=['cid', 'genero'])
comments = comments.merge(genero_df)

comments_by_channel_F = []
comments_by_channel_M = []
for channel, group in comments.groupby('uploader'):
    channel_comments = ' '.join(group[group['genero']=='F']['text'])
    comments_by_channel_F.append(channel_comments)
    channel_comments = ' '.join(group[group['genero']=='M']['text'])
    comments_by_channel_M.append(channel_comments)

In [12]:
X, labels = word_occurence_matrix(comments_by_channel_F, stop_words=stop_words, binary=False)
#Makes frequencies relative to each channel so that we have the probability distribution of words for each channel
X = (X.T / X.sum(axis=1)).T

freq_lists = [list(zip(['%.4f'%(freq) for freq in row],labels)) for row in X]
freq_lists = [sorted(channel, reverse=True) for channel in freq_lists]
df_freq = pd.DataFrame(freq_lists, index = channels).T

print('Palavras com maior frequência relativa entre mulheres em cada canal:')
df_freq.head(10)

Palavras com maior frequência relativa entre mulheres em cada canal:


,Dr. Alain Dutra,Dr. Alvaro Galvão,Dr. Felipe Ades MD PhD,Dr. Fernando Gomes,Dr. Lair Ribeiro Oficial,Drauzio Varella,Julio Pereira - Neurocirurgião,Lucy Kerr
0,"(0.0187, dias)","(0.0263, dias)","(0.0334, dias)","(0.0744, dr)","(0.0341, dr)","(0.0132, dr)","(0.0226, dias)","(0.0160, ivermectina)"
1,"(0.0156, tomar)","(0.0188, dr)","(0.0176, sintomas)","(0.0236, noite)","(0.0103, deus)","(0.0079, virus)","(0.0161, deus)","(0.0136, dias)"
2,"(0.0140, ivermectina)","(0.0170, tomar)","(0.0164, covid)","(0.0163, parabens)","(0.0093, virus)","(0.0078, video)","(0.0158, olfato)","(0.0128, dra)"
3,"(0.0137, dr)","(0.0156, 15)","(0.0147, deu)","(0.0080, obrigada)","(0.0069, gratidao)","(0.0078, pessoas)","(0.0154, paladar)","(0.0102, tomar)"
4,"(0.0130, deus)","(0.0152, ivermectina)","(0.0142, dor)","(0.0069, doutor)","(0.0065, obrigada)","(0.0067, casa)","(0.0151, cheiro)","(0.0102, deus)"
5,"(0.0093, obrigada)","(0.0127, comprimidos)","(0.0142, dia)","(0.0065, dia)","(0.0052, grande)","(0.0064, obrigada)","(0.0136, dor)","(0.0069, dr)"
6,"(0.0086, dia)","(0.0098, obrigada)","(0.0136, deus)","(0.0065, deus)","(0.0050, abencoe)","(0.0063, dia)","(0.0109, sinto)","(0.0067, 15)"
7,"(0.0081, comprimidos)","(0.0082, dose)","(0.0130, teste)","(0.0065, amor)","(0.0048, gente)","(0.0063, deus)","(0.0094, sintomas)","(0.0065, covid)"
8,"(0.0078, pode)","(0.0079, deus)","(0.0119, positivo)","(0.0054, excelente)","(0.0048, doutor)","(0.0061, dias)","(0.0092, gosto)","(0.0055, pode)"
9,"(0.0068, tomei)","(0.0073, doutor)","(0.0085, muita)","(0.0047, adorei)","(0.0047, saude)","(0.0055, senhor)","(0.0091, sentir)","(0.0053, pessoas)"


In [10]:
X, labels = word_occurence_matrix(comments_by_channel_M, stop_words=stop_words, binary=False)
#Makes frequencies relative to each channel so that we have the probability distribution of words for each channel
X = (X.T / X.sum(axis=1)).T

freq_lists = [list(zip(['%.4f'%(freq) for freq in row],labels)) for row in X]
freq_lists = [sorted(channel, reverse=True) for channel in freq_lists]
df_freq = pd.DataFrame(freq_lists, index = channels).T

print('Palavras com maior frequência relativa entre homens em cada canal:')
df_freq.head(10)

Palavras com maior frequência relativa entre homens em cada canal:


,Dr. Alain Dutra,Dr. Alvaro Galvão,Dr. Felipe Ades MD PhD,Dr. Fernando Gomes,Dr. Lair Ribeiro Oficial,Drauzio Varella,Julio Pereira - Neurocirurgião,Lucy Kerr
0,"(0.0229, dias)","(0.0140, ivermectina)","(0.0247, dor)","(0.0184, sr)","(0.0230, dr)","(0.0092, virus)","(0.0205, dias)","(0.0175, ivermectina)"
1,"(0.0149, comprimidos)","(0.0126, dr)","(0.0234, dia)","(0.0161, parabens)","(0.0130, virus)","(0.0088, video)","(0.0145, cheiro)","(0.0106, dias)"
2,"(0.0138, tomar)","(0.0119, dias)","(0.0208, dias)","(0.0138, mundo)","(0.0079, saude)","(0.0082, dr)","(0.0141, olfato)","(0.0098, dra)"
3,"(0.0132, ivermectina)","(0.0088, parabens)","(0.0169, virus)","(0.0138, entrevista)","(0.0056, video)","(0.0071, pessoas)","(0.0134, paladar)","(0.0077, comprimidos)"
4,"(0.0119, kg)","(0.0087, tomar)","(0.0143, sintomas)","(0.0115, mundial)","(0.0053, pessoas)","(0.0046, senhor)","(0.0117, deus)","(0.0071, tomar)"
5,"(0.0093, dr)","(0.0064, pessoas)","(0.0143, positivo)","(0.0115, farsa)","(0.0048, deus)","(0.0044, pode)","(0.0096, sentir)","(0.0058, covid)"
6,"(0.0079, pode)","(0.0064, dose)","(0.0143, pode)","(0.0092, china)","(0.0046, senhor)","(0.0044, casa)","(0.0093, gosto)","(0.0054, deus)"
7,"(0.0079, durante)","(0.0062, 15)","(0.0143, deu)","(0.0092, brasil)","(0.0044, mundo)","(0.0043, doutor)","(0.0087, dia)","(0.0053, dr)"
8,"(0.0071, covid)","(0.0059, covid)","(0.0143, covid)","(0.0069, virus)","(0.0044, china)","(0.0040, dia)","(0.0082, sinto)","(0.0052, pode)"
9,"(0.0070, tratamento)","(0.0057, doutor)","(0.0117, exame)","(0.0069, vamos)","(0.0041, covid)","(0.0040, brasil)","(0.0079, covid)","(0.0051, tratamento)"
